<a href="https://colab.research.google.com/github/GustavinhoDantas/Verificacao-duplicidades/blob/main/Duplicidade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DUPLICIDADES



In [1]:
import pandas as pd

# Caminhos dos arquivos
arquivo_subir = '/content/Subir_IBI_BV.xlsx'
arquivo_api = '/content/Lote_258_2025_06_11_10_32_52.xlsx'
arquivo_portal = '/content/query_result_2025-06-11T11_48_14.593598-03_00.xlsx'

arq_subir = pd.read_excel(arquivo_subir)
arq_api = pd.read_excel(arquivo_api)
arq_portal = pd.read_excel(arquivo_portal)
print('lidas')

/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


lidas


In [2]:
# Acesso seguro às colunas com strip
arq_subir['Apólice'] = arq_subir['Apólice'].str.strip()
arq_api['Número da Apólice'] = arq_api['Número da Apólice'].str.strip()
arq_portal['Apólice'] = arq_portal['Apólice'].str.strip()
arq_portal['Apólice renovada'] = arq_portal['Apólice renovada'].str.strip()


#print(arq_subir['Apólice'])

In [ ]:
# Vendo se a apolice em questão já não está no portal, batendo apolice x apolice e apolice  x endosso ]


0     02-0775-1047029
1     02-0775-1187131
2     02-0775-1179866
3     02-0775-1187451
4     02-0775-1167083
5     02-0775-1175677
6     02-0775-1175109
7     02-0775-1228572
9     02-0775-1187435
10    02-0775-1196717
11    02-0775-1171395
12    02-0775-1230605
13    02-0775-1230723
14    02-0775-1220014
15    02-0775-0809733
17    02-0775-1195306
18    02-0775-1171017
19    02-0775-1163466
20    02-0775-1046850
21    02-0775-1179870
22    02-0775-1183045
23    02-0775-1174924
24    02-0775-1179869
25    02-0775-1187446
26    02-0775-1164218
27    02-0775-1175679
28    02-0775-1187437
29    02-0775-1174926
30    02-0775-1187445
31    02-0775-1164794
32    02-0775-1178849
33    02-0775-1175108
34    02-0775-1171534
35    02-0775-0840353
36    02-0775-1183938
37    02-0775-1163467
38    02-0775-1187433
39    02-0775-1175674
40    02-0775-1164217
41    02-0775-1171396
42    02-0775-1173015
43    02-0775-1189487
44    02-0775-1166448
45    02-0775-1187301
46    02-0775-1003217
47    02-0

In [3]:
# Criar uma máscara booleana para linhas da arq_api onde a apólice está em arq_subir['Apólice']
mask_api = arq_api['Número da Apólice'].isin(arq_subir['Apólice'])
correspondentes_api = arq_api.loc[mask_api]

print('Vendo correspondencias')

#df = pd.DataFrame(correspondentes)
#df.to_excel('teste.xlsx', index=False)
#print('Dados extraídos e salvos no Excel!')

Vendo correspondencias


In [7]:
#Pegando a correspondencia, mas apenas que são renovações
renovacao = correspondentes_api[correspondentes_api['Renovação?'] == 'sim']
print('Vista apenas as renovações')

#tira a repetição da mesma apolice
renovacao = renovacao.drop_duplicates(subset='Número da Apólice', keep='first')
print('duplicidades')

cancelados = renovacao[renovacao['Status'] == 'cancelado']
renovacao = renovacao[renovacao['Status'] != 'cancelado']

df = pd.DataFrame(renovacao)
cancelados.to_excel('cancelados.xlsx', index=False)
df.to_excel('renovacao.xlsx', index=False)
print('Dados extraídos e salvos no Excel!')

Vista apenas as renovações
duplicidades
Dados extraídos e salvos no Excel!


Agora fazendo o bate com o portal

In [ ]:
# Garantir que os valores estejam padronizados como strings e sem espaços
renovacao['Número da Apólice Renovada'] = renovacao['Número da Apólice Renovada'].astype(str).str.strip()
arq_portal['Apólice'] = arq_portal['Apólice'].astype(str).str.strip()
arq_portal['Apólice renovada'] = arq_portal['Apólice renovada'].astype(str).str.strip()

# Máscara correta: procurar na tabela portal os valores que estão na renovacao
mask_portal = (
    arq_portal['Apólice'].isin(renovacao['Número da Apólice Renovada']) |
    arq_portal['Apólice renovada'].isin(renovacao['Número da Apólice Renovada']) |
    arq_portal['Endosso'].isin(renovacao['Número da Apólice Renovada'])
)
# Filtrar as linhas da tabela do portal
correspondentes_portal = arq_portal[mask_portal]

# Salvar no Excel
correspondentes_portal.to_excel('correspondencia_portal.xlsx', index=False)
print('Linhas do portal que batem com renovações salvas no Excel!')


Linhas do portal que batem com renovações salvas no Excel!



- Bater a SUBIR já no portal (apolice x apolice e apolice x endosso) - fazer com que retorne uma tabela com campus ID portal|Apolice|Endosso
- Maneira de bater a planilha renovacao com a do portal para ver duplicidades, para isso é necessario ver a trilha (trilha é aquela que tem vinculo com a mesma apolice, ou seja, tudo que tiver na base do portal com a coluna "Apolice" igual é a trilha) para ver isso é necessario bater as datas, se a data de final de vigencia da trilha for igual a da apolice tudo bem, caso seja diferente houve uma renovação, por isso é uma duplicidade

